In [73]:
import wandb
import os

In [74]:
RUN_PATH = "team-epoch-iv/detect-harmful-brain-activity/us9xl2nw"
#"team-epoch-iv/detect-harmful-brain-activity/apx45m70" 
# "team-epoch-iv/detect-harmful-brain-activity/pq7jg3r1"

In [75]:
api = wandb.Api()
run = api.run(f"{RUN_PATH}")

In [76]:
is_sweep_run = run.sweep is not None
print(f"Is sweep run: {is_sweep_run}")

Is sweep run: False


In [77]:
# list all artefacts
model_artefacts = []
for artefact in run.logged_artifacts():
    if artefact.type == "model":
        print(artefact.name)
        model_artefacts.append(artefact)

Chris-512:v523
Chris-512:v525
Chris-512:v529
Chris-512:v530
Chris-512:v536
Chris-512:v537
Chris-512:v541
Chris-512:v543


In [78]:
# download all artefacts
for artefact in model_artefacts:
    for file in artefact.files():
        print(f"Downloading {file.name} from {artefact.name}")
        file.download(root="../downloads", exist_ok=True)

In [79]:
# get the config type artefact (should be only one)
config_artefact = None
for artefact in run.logged_artifacts():
    if artefact.type == "config":
        config_artefact = artefact
        break

In [80]:
if not is_sweep_run:
    # download the raw, human readable config file
    for file in config_artefact.files():
        if file.name not in ["config.yaml","cv.yaml","train.yaml"]:
            print(f"Downloading {file.name} from {config_artefact.name}")
            file.download(root="../downloads", replace=True)
            # rename the file to the name of the run
            os.rename(f"../downloads/{file.name}", f"../downloads/{run.name}")
            print(f"Renamed to {run.name}")

Renamed to persistent-inning


In [81]:
from omegaconf import OmegaConf

if is_sweep_run:
    # download the file called config.yaml from the artifact, which includes sweep parameters
    for file in config_artefact.files():
        if file.name == "config.yaml":
            print(f"Downloading {file.name} from {config_artefact.name}")
            file.download(root="../downloads", replace=True)
            
            # read the file into a dictconfig with omegaconf
            config = OmegaConf.load(f"../downloads/{file.name}")        
            config = config.model
            # save the config again
            OmegaConf.save(config, f"../downloads/{file.name}")        
            
            # rename the file to the name of the run
            os.rename(f"../downloads/{file.name}", f"../downloads/{run.name}.yaml")
            print(f"Renamed to {run.name}.yaml")
